**Note: This notebook must be changed by the student! As it is now, it does not approximate the solution to the Poisson problem.**

# 2D Poisson Solver on the Unit Square

In the exercise we write a solver for the 2D Poisson problem on the unit square using Psydac's bilinear form interface.

\begin{align}
    -\Delta\phi &= f \quad \text{ in }\Omega=]0,1[^2, \\
    \phi &= 0 \quad \text{ on }\partial\Omega.
\end{align}

## The Variational Formulation

The corresponding variational formulation reads

\begin{align}
    \text{Find }\phi\in V\coloneqq H_0^1(\Omega)\text{ such that } \\
    a(\phi, \psi) = l(\psi)\quad\forall\ \psi\in V
\end{align}

where 
- $a(\phi,\psi) \coloneqq \int_{\Omega} \nabla\phi\cdot\nabla\psi ~ d\Omega$,
- $l(\psi) := \int_{\Omega} f\psi ~ d\Omega$.

## Formal Model

In [ ]:
from sympde.calculus    import dot, grad
from sympde.expr        import BilinearForm, LinearForm, integral
from sympde.topology    import elements_of, Square
from sympde.topology    import ScalarFunctionSpace

from sympy import pi, sin

domain  = Square('S', bounds1=(0, 1), bounds2=(0, 1))

V       = ScalarFunctionSpace('V', domain)
x,y     = domain.coordinates

phi, psi    = elements_of(V, names='phi, psi')

# bilinear form
a       = BilinearForm((phi, psi), integral(domain, phi*psi))

# linear form
f       = 8 * (pi**2) * sin(2 * pi * x) * sin(2 * pi * y)

l       = LinearForm(psi, integral(domain, f*psi))

## Discretization

We will use Psydac to discretize the problem.

In [ ]:
from psydac.api.discretization  import discretize

In [ ]:
ncells  = [8, 8]    # Bspline cells
degree  = [2, 2]    # Bspline degree

In [ ]:
from psydac.api.settings        import PSYDAC_BACKEND_GPYCCEL

backend     = PSYDAC_BACKEND_GPYCCEL

domain_h    = discretize(domain, ncells=ncells, periodic=[False, False])
V_h         = discretize(V, domain_h, degree=degree)

a_h         = discretize(a, domain_h, (V_h, V_h), backend=backend)
l_h         = discretize(l, domain_h, V_h, backend=backend)

## Boundary Conditions

We choose to apply a projection method. For that matter, we construct the projection matrix $\mathbb{P}_0$ and its counterpart $\mathbb{P}_{\Gamma} = \mathbb{I} - \mathbb{P}_0$.

In [ ]:
from utils import H1BoundaryProjector2D
from psydac.linalg.basic import IdentityOperator

P_0     = H1BoundaryProjector2D_2(V, V_h.vector_space)

I_0     = IdentityOperator(V_h.vector_space)
P_Gamma = I_0 - P_0

In [ ]:
A       = a_h.assemble()
f       = l_h.assemble()

A_bc    = P_0 @ A @ P_0 + P_Gamma
f_bc    = P_0 @ f

## Solving the PDE

In [ ]:
from psydac.linalg.solvers import inverse

tol     = 1e-12
maxiter = 1000

phi_h   = inverse(A_bc, 'cg', tol=tol, maxiter=maxiter) @ f_bc

## Computing the error norm

When the analytical solution is available, you might be interested in computing the $L^2$ norm or $H^1_0$ semi-norm.
SymPDE allows you to do so, by creating the **Norm** object.
In this example, the analytical solution is given by

$$
phi_{ex}(x, y) = \sin(2\pi x)\sin(2\pi y)
$$

### Computing the $L^2$ norm

In [ ]:
from sympde.expr     import find, EssentialBC, Norm, SemiNorm
from psydac.fem.basic import FemField

phi_ex  = sin(2*pi*x) * sin(2*pi*y)

error = phi_ex - phi

# create the formal Norm object
l2norm = Norm(error, domain, kind='l2')

# discretize the norm
l2norm_h = discretize(l2norm, domain_h, V_h)

# assemble the norm
l2_error = l2norm_h.assemble(phi=FemField(V_h, phi_h))

# print the result
print(l2_error)

### Computing the $H^1$ semi-norm

In [ ]:
# create the formal Norm object
h1norm = SemiNorm(error, domain, kind='h1')

# discretize the norm
h1norm_h = discretize(h1norm, domain_h, V_h)

# assemble the norm
h1_error = h1norm_h.assemble(phi=FemField(V_h, phi_h))

# print the result
print(h1_error)

### Computing the $H^1$ norm

In [ ]:
# create the formal Norm object
h1norm = Norm(error, domain, kind='h1')

# discretize the norm
h1norm_h = discretize(h1norm, domain_h, V_h)

# assemble the norm
h1_error = h1norm_h.assemble(phi=FemField(V_h, phi_h))

# print the result
print(h1_error)

## Visualization

We plot the true solution $\phi_{ex}$, the approximate solution $\phi_h$ and the error function $|\phi_{ex} - \phi_h|$.

In [ ]:
from utils import plot
from sympy import lambdify

phi_ex_fun = lambdify(domain.coordinates, phi_ex)
phi_h_fun = FemField(V_h, phi_h)
error = lambda x, y: abs(phi_ex_fun(x, y) - phi_h_fun(x, y))

plot(gridsize_x   = 100, 
    gridsize_y    = 100, 
    title         = r'Approximation of Solution $\phi$', 
    funs          = [phi_ex_fun, phi_h_fun, error], 
    titles        = [r'$\phi_{ex}(x,y)$', r'$\phi_h(x,y)$', r'$|(\phi_{ex}-\phi_h)(x,y)|$'],
    surface_plot  = True
)